In [12]:
import os

import xarray as xr
import pandas as pd
import numpy as np
import seaborn as sns
import dask.dataframe as dd

import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

import dash 
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [13]:
#Importar data .parquet

carpeta = r"G:\My Drive\UTFSM\memoria\tareas\resul_series_sce400\parquet_sp_b"

files = os.path.join(carpeta, '*sp.parquet')
print(files)

#dataframe con todos los datos
df = dd.read_parquet(files).reset_index()

#arregla tipo de datos (str o float)
df['magnitud']=df['magnitud'].astype(str)
df[['escenario','grid_npoints']]=df[['escenario','grid_npoints']].astype(int) #grid_npoints

df[['longitude','latitude','T','S']]=df[['longitude','latitude','T','S']
                      ].astype(float)


df=df.compute()


G:\My Drive\UTFSM\memoria\tareas\resul_series_sce400\parquet_sp_b\*sp.parquet


In [14]:
archivo_xy=r"G:\My Drive\UTFSM\memoria\avances\Python\Detalle_boyas_v2.txt"
data_xy=pd.read_csv(archivo_xy,sep='\t'
                   )

del data_xy['scale']
del data_xy['ge_hotspot']
del data_xy['altitude (m)']
del data_xy['sym']
del data_xy['type']
data_xy.insert(0, 'code', range(0, 131))


data_xy.head()

,code,latitude,longitude,name
0,0,-29.937247,288.663768,Boya0
1,1,-29.946242,288.663434,Boya1
2,2,-29.950016,288.663113,Boya2
3,3,-29.950803,288.663332,Boya3
4,4,-29.950987,288.662254,Boya4


In [15]:
token = open(r"C:\Users\mau\OneDrive\Desktop\token.mapbox_token").read() # you will need your own token

fig = px.scatter_mapbox(data_xy, lat='latitude', lon='longitude', hover_name="code", hover_data=["latitude", "longitude"],
                            color_discrete_sequence=["fuchsia"], zoom=3, height=700)
#fig.update_layout(
#    mapbox_style="white-bg",
#    mapbox_layers=[
#        {
#            "below": 'traces',
#            "sourcetype": "raster",
#            "sourceattribution": "United States Geological Survey",
#            "source": [
#                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
#            ]
#        }
#      ])

fig.update_layout(mapbox_style="light", mapbox_accesstoken=token)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})



In [16]:
magnitudes={0: 'mw_8.6',
           1: 'mw_8.7',
           2:'mw_8.8',
           3:'mw_8.9',
           4:'mw_9.0',
           5:'mw_9.1',
           6:'mw_9.2'}

In [17]:
#App layout
app.layout = html.Div([

    html.H1("Espectrogramas bahías chilenas", style={'text-align': 'center'}),

    html.Div(
        dcc.Slider(
           id='crossfilter-mw-slider',
           min=0,
           max=6,
           value=0,
           marks=magnitudes,
           step=None
        ), style={'width': '100%', 'padding': '0px 20px 20px 20px'}),
    
    html.Div([
        dcc.Graph(id='crossfilter-indicator-scatter',
                  figure=fig,
                  hoverData={'points': [{'hovertext': 0}]}
        )
    ], style={'width': '49%','height':'700px', 'display': 'inline-block','margin-left':'20px'}),        
    
    html.Div([
            dcc.Graph(id='espectrograma'),
            dcc.Graph(id='heatmap')
        ], style={'display': 'inline-block', 'width': '49%'}),

])

In [18]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [dash.dependencies.Output('espectrograma', 'figure'),
     dash.dependencies.Output('heatmap', 'figure')],
    [dash.dependencies.Input('crossfilter-mw-slider', 'value'),
    dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData')]
)


def update_esp(option_slctd,hoverData):
    dff=df[df.magnitud == magnitudes[option_slctd]]
    boya=hoverData['points'][0]['hovertext']
    dff=dff[dff.grid_npoints==int(boya)]
    
    
    layout = {"scene": {"xaxis": { "title": "Periodo"}, 
        "yaxis": {"title": "Escenario"}, 
        "zaxis": {"type": "log", "title": "PSD"} }
              ,"legend": {"y": 0.5, "yanchor": "top"}, 
      "margin": {"b": 40, "l": 60, "r": 10, "t": 25}}
    z=dff['S']/dff['S'].max()
    x=dff['T']
    y=dff['escenario']
    
    fig=go.Figure(data=[go.Mesh3d(x=x,y=y,z=z, colorbar_title='PSD'
                                  ,opacity=1, intensity=z, colorscale='YlOrRd')],layout=layout)
    fig.update_layout(title='Espectrograma boya', width=500,height=400)
    
     #ordena los z y los deja como matriz
    dfp=pd.pivot_table(dff,values='S', index='T',columns='escenario')
    
    #x,y,z para los heatmap
    escenarios=dfp.columns.values
    periodos=dfp.index.values
    Z=dfp.values
    
    # escala lineal
    fig2=go.Figure(data=[go.Heatmap(z=Z,
                                   x=escenarios,
                                   y=periodos,
                                   hovertemplate='Escenario: %{x} <br>Periodo: %{y:.3f} <br>PSD: %{z:.3f}<extra></extra>',
                                   #'<b>z1:%{z:.3f}</b><br>z2:%{customdata[0]:.3f} <br>z3: %{customdata[1]:.3f} '
                                   colorscale='jet',
                                   colorbar=dict(
                                       ticks="outside"
                                   )
                                   
                                  )
                       ]
                 )
    fig2.update_yaxes(type='log')
    fig2.update_layout(transition_duration=500,width=500,height=400)
    return fig,fig2




In [19]:
app.run_server(debug=True,use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
